In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import warnings
import time
import sklearn.preprocessing

pd.set_option("display.max_columns", None)
warnings.filterwarnings(action='ignore')

serviceKey='bwUUK6MEA6GnqDpo2ZS9ChN4VZ1nkUPXrryU80XxUcLqxB62pIGIPWXNdyzqgT20BHrw8zxWWk9rqiSQwTrbnQ=='

p7000 ={'serviceKey' :serviceKey,
             'routeId':'200000112','stationId':'228000703'}
class Bus():
    def __init__(self,params):
        self.params=params
        self.bus_time=[]
        
    def repet(self):
        url = 'http://apis.data.go.kr/6410000/busarrivalservice/getBusArrivalItem'
        response = requests.get(url, params=self.params)
        soup=BeautifulSoup(response.text,'xml')
        return soup

        
    def current_buses(self): 
        url = 'http://apis.data.go.kr/6410000/buslocationservice/getBusLocationList'
        mod_params ={'serviceKey' : self.params['serviceKey'],
                  'routeId':self.params['routeId']}

        response=requests.get(url,params=mod_params)
        soup=BeautifulSoup(response.text,'xml')
        locations=sorted([int(x.stationSeq.text) for x in soup.find_all('busLocationList')])
        return locations


def trim_data(df,FileName):
    df['시간'] = df['시간'].apply(pd.to_datetime)
    df['hour']=df['시간'].dt.hour
    df['minute']=df['시간'].dt.minute
    df['second']=df['시간'].dt.second
    df['day'] = df['시간'].dt.weekday 

    mb = sklearn.preprocessing.MultiLabelBinarizer()
    if isinstance(df['운행중인 다른 버스들의 위치'].iloc[0], str):
        df['운행중인 다른 버스들의 위치'] = df['운행중인 다른 버스들의 위치'].apply(eval)
    entity = mb.fit_transform(df['운행중인 다른 버스들의 위치']) 
    lsts=pd.DataFrame(entity,columns=mb.classes_)
    lsts.columns=[f'station{x}' for x in lsts.columns]
    df=pd.concat([df,lsts],axis=1,sort=False).reset_index(drop=True)


    df=df.drop(["운행중인 다른 버스들의 위치"],axis=1)
    
    batch=df.index[df.arrived==1]
    df=df.iloc[:batch[-1]+1,:]
    #7/12 -----------------------------------
    batch=df.index[df.arrived==1]
    temp=0
    for x in batch:
        for y in range(temp,x): 
            df['arrived'][y]=(df['시간'][x]-df['시간'][y]).seconds
            temp=x+1
    for e in batch:
        df['arrived'][e]=0
    # 7/12 -----------------------------------

    df = df.set_index('시간')
    df.to_csv(f'./{FileName}')        # 동일 폴더에 파일이 저장될 거임
    return df

In [3]:
past=4
pBus=Bus(p7000)
for x in range(720): # 시간 설정
    soup=pBus.repet()
    try:
        result=int(soup.resultCode.text)
        arrived=0
        if past!=result:
            if result==0:
                arrived=1
        pBus.bus_time.append([soup.queryTime.text,pBus.current_buses(),arrived])
        past=result
        print(x,"번째 시도: ",pBus.bus_time)
        time.sleep(60)
    except Exception as e:
        continue

df=pd.DataFrame(pBus.bus_time,columns=['시간','운행중인 다른 버스들의 위치','arrived'])

mod_df=trim_data(df,'8.11.csv') # 이름 설정
mod_df

0 번째 시도:  [['2022-08-11 10:35:37.080', [11, 22, 32, 42, 50, 52, 60, 68], 0]]
1 번째 시도:  [['2022-08-11 10:35:37.080', [11, 22, 32, 42, 50, 52, 60, 68], 0], ['2022-08-11 10:36:37.320', [11, 23, 32, 43, 51, 52, 60, 69], 0]]
2 번째 시도:  [['2022-08-11 10:35:37.080', [11, 22, 32, 42, 50, 52, 60, 68], 0], ['2022-08-11 10:36:37.320', [11, 23, 32, 43, 51, 52, 60, 69], 0], ['2022-08-11 10:37:38.482', [12, 24, 33, 44, 51, 52, 61, 69], 0]]
3 번째 시도:  [['2022-08-11 10:35:37.080', [11, 22, 32, 42, 50, 52, 60, 68], 0], ['2022-08-11 10:36:37.320', [11, 23, 32, 43, 51, 52, 60, 69], 0], ['2022-08-11 10:37:38.482', [12, 24, 33, 44, 51, 52, 61, 69], 0], ['2022-08-11 10:38:38.661', [12, 24, 33, 44, 51, 52, 62, 70], 0]]
4 번째 시도:  [['2022-08-11 10:35:37.080', [11, 22, 32, 42, 50, 52, 60, 68], 0], ['2022-08-11 10:36:37.320', [11, 23, 32, 43, 51, 52, 60, 69], 0], ['2022-08-11 10:37:38.482', [12, 24, 33, 44, 51, 52, 61, 69], 0], ['2022-08-11 10:38:38.661', [12, 24, 33, 44, 51, 52, 62, 70], 0], ['2022-08-11 10:39:39

16 번째 시도:  [['2022-08-11 10:35:37.080', [11, 22, 32, 42, 50, 52, 60, 68], 0], ['2022-08-11 10:36:37.320', [11, 23, 32, 43, 51, 52, 60, 69], 0], ['2022-08-11 10:37:38.482', [12, 24, 33, 44, 51, 52, 61, 69], 0], ['2022-08-11 10:38:38.661', [12, 24, 33, 44, 51, 52, 62, 70], 0], ['2022-08-11 10:39:39.792', [13, 25, 34, 45, 51, 52, 63, 70], 0], ['2022-08-11 10:40:40.001', [14, 26, 34, 45, 52, 52, 63, 72], 0], ['2022-08-11 10:41:40.117', [3, 14, 26, 36, 46, 52, 53, 63, 74], 0], ['2022-08-11 10:42:43.357', [3, 15, 26, 36, 46, 52, 53, 64, 74], 0], ['2022-08-11 10:43:43.550', [4, 16, 26, 37, 47, 52, 53, 65, 74], 0], ['2022-08-11 10:44:46.759', [5, 16, 27, 37, 48, 52, 54, 65, 75], 0], ['2022-08-11 10:45:47.952', [6, 17, 27, 38, 48, 52, 54, 66, 77], 0], ['2022-08-11 10:46:48.099', [7, 18, 27, 39, 49, 53, 55, 68, 78], 0], ['2022-08-11 10:47:48.402', [8, 18, 27, 39, 49, 53, 55, 68, 79], 0], ['2022-08-11 10:48:48.548', [9, 18, 28, 40, 50, 53, 55, 69], 0], ['2022-08-11 10:49:49.820', [9, 19, 28, 41, 

23 번째 시도:  [['2022-08-11 10:35:37.080', [11, 22, 32, 42, 50, 52, 60, 68], 0], ['2022-08-11 10:36:37.320', [11, 23, 32, 43, 51, 52, 60, 69], 0], ['2022-08-11 10:37:38.482', [12, 24, 33, 44, 51, 52, 61, 69], 0], ['2022-08-11 10:38:38.661', [12, 24, 33, 44, 51, 52, 62, 70], 0], ['2022-08-11 10:39:39.792', [13, 25, 34, 45, 51, 52, 63, 70], 0], ['2022-08-11 10:40:40.001', [14, 26, 34, 45, 52, 52, 63, 72], 0], ['2022-08-11 10:41:40.117', [3, 14, 26, 36, 46, 52, 53, 63, 74], 0], ['2022-08-11 10:42:43.357', [3, 15, 26, 36, 46, 52, 53, 64, 74], 0], ['2022-08-11 10:43:43.550', [4, 16, 26, 37, 47, 52, 53, 65, 74], 0], ['2022-08-11 10:44:46.759', [5, 16, 27, 37, 48, 52, 54, 65, 75], 0], ['2022-08-11 10:45:47.952', [6, 17, 27, 38, 48, 52, 54, 66, 77], 0], ['2022-08-11 10:46:48.099', [7, 18, 27, 39, 49, 53, 55, 68, 78], 0], ['2022-08-11 10:47:48.402', [8, 18, 27, 39, 49, 53, 55, 68, 79], 0], ['2022-08-11 10:48:48.548', [9, 18, 28, 40, 50, 53, 55, 69], 0], ['2022-08-11 10:49:49.820', [9, 19, 28, 41, 

28 번째 시도:  [['2022-08-11 10:35:37.080', [11, 22, 32, 42, 50, 52, 60, 68], 0], ['2022-08-11 10:36:37.320', [11, 23, 32, 43, 51, 52, 60, 69], 0], ['2022-08-11 10:37:38.482', [12, 24, 33, 44, 51, 52, 61, 69], 0], ['2022-08-11 10:38:38.661', [12, 24, 33, 44, 51, 52, 62, 70], 0], ['2022-08-11 10:39:39.792', [13, 25, 34, 45, 51, 52, 63, 70], 0], ['2022-08-11 10:40:40.001', [14, 26, 34, 45, 52, 52, 63, 72], 0], ['2022-08-11 10:41:40.117', [3, 14, 26, 36, 46, 52, 53, 63, 74], 0], ['2022-08-11 10:42:43.357', [3, 15, 26, 36, 46, 52, 53, 64, 74], 0], ['2022-08-11 10:43:43.550', [4, 16, 26, 37, 47, 52, 53, 65, 74], 0], ['2022-08-11 10:44:46.759', [5, 16, 27, 37, 48, 52, 54, 65, 75], 0], ['2022-08-11 10:45:47.952', [6, 17, 27, 38, 48, 52, 54, 66, 77], 0], ['2022-08-11 10:46:48.099', [7, 18, 27, 39, 49, 53, 55, 68, 78], 0], ['2022-08-11 10:47:48.402', [8, 18, 27, 39, 49, 53, 55, 68, 79], 0], ['2022-08-11 10:48:48.548', [9, 18, 28, 40, 50, 53, 55, 69], 0], ['2022-08-11 10:49:49.820', [9, 19, 28, 41, 

33 번째 시도:  [['2022-08-11 10:35:37.080', [11, 22, 32, 42, 50, 52, 60, 68], 0], ['2022-08-11 10:36:37.320', [11, 23, 32, 43, 51, 52, 60, 69], 0], ['2022-08-11 10:37:38.482', [12, 24, 33, 44, 51, 52, 61, 69], 0], ['2022-08-11 10:38:38.661', [12, 24, 33, 44, 51, 52, 62, 70], 0], ['2022-08-11 10:39:39.792', [13, 25, 34, 45, 51, 52, 63, 70], 0], ['2022-08-11 10:40:40.001', [14, 26, 34, 45, 52, 52, 63, 72], 0], ['2022-08-11 10:41:40.117', [3, 14, 26, 36, 46, 52, 53, 63, 74], 0], ['2022-08-11 10:42:43.357', [3, 15, 26, 36, 46, 52, 53, 64, 74], 0], ['2022-08-11 10:43:43.550', [4, 16, 26, 37, 47, 52, 53, 65, 74], 0], ['2022-08-11 10:44:46.759', [5, 16, 27, 37, 48, 52, 54, 65, 75], 0], ['2022-08-11 10:45:47.952', [6, 17, 27, 38, 48, 52, 54, 66, 77], 0], ['2022-08-11 10:46:48.099', [7, 18, 27, 39, 49, 53, 55, 68, 78], 0], ['2022-08-11 10:47:48.402', [8, 18, 27, 39, 49, 53, 55, 68, 79], 0], ['2022-08-11 10:48:48.548', [9, 18, 28, 40, 50, 53, 55, 69], 0], ['2022-08-11 10:49:49.820', [9, 19, 28, 41, 

...

165 번째 시도:  [['2022-08-11 10:35:37.080', [11, 22, 32, 42, 50, 52, 60, 68], 0], ['2022-08-11 10:36:37.320', [11, 23, 32, 43, 51, 52, 60, 69], 0], ['2022-08-11 10:37:38.482', [12, 24, 33, 44, 51, 52, 61, 69], 0], ['2022-08-11 10:38:38.661', [12, 24, 33, 44, 51, 52, 62, 70], 0], ['2022-08-11 10:39:39.792', [13, 25, 34, 45, 51, 52, 63, 70], 0], ['2022-08-11 10:40:40.001', [14, 26, 34, 45, 52, 52, 63, 72], 0], ['2022-08-11 10:41:40.117', [3, 14, 26, 36, 46, 52, 53, 63, 74], 0], ['2022-08-11 10:42:43.357', [3, 15, 26, 36, 46, 52, 53, 64, 74], 0], ['2022-08-11 10:43:43.550', [4, 16, 26, 37, 47, 52, 53, 65, 74], 0], ['2022-08-11 10:44:46.759', [5, 16, 27, 37, 48, 52, 54, 65, 75], 0], ['2022-08-11 10:45:47.952', [6, 17, 27, 38, 48, 52, 54, 66, 77], 0], ['2022-08-11 10:46:48.099', [7, 18, 27, 39, 49, 53, 55, 68, 78], 0], ['2022-08-11 10:47:48.402', [8, 18, 27, 39, 49, 53, 55, 68, 79], 0], ['2022-08-11 10:48:48.548', [9, 18, 28, 40, 50, 53, 55, 69], 0], ['2022-08-11 10:49:49.820', [9, 19, 28, 41,

166 번째 시도:  [['2022-08-11 10:35:37.080', [11, 22, 32, 42, 50, 52, 60, 68], 0], ['2022-08-11 10:36:37.320', [11, 23, 32, 43, 51, 52, 60, 69], 0], ['2022-08-11 10:37:38.482', [12, 24, 33, 44, 51, 52, 61, 69], 0], ['2022-08-11 10:38:38.661', [12, 24, 33, 44, 51, 52, 62, 70], 0], ['2022-08-11 10:39:39.792', [13, 25, 34, 45, 51, 52, 63, 70], 0], ['2022-08-11 10:40:40.001', [14, 26, 34, 45, 52, 52, 63, 72], 0], ['2022-08-11 10:41:40.117', [3, 14, 26, 36, 46, 52, 53, 63, 74], 0], ['2022-08-11 10:42:43.357', [3, 15, 26, 36, 46, 52, 53, 64, 74], 0], ['2022-08-11 10:43:43.550', [4, 16, 26, 37, 47, 52, 53, 65, 74], 0], ['2022-08-11 10:44:46.759', [5, 16, 27, 37, 48, 52, 54, 65, 75], 0], ['2022-08-11 10:45:47.952', [6, 17, 27, 38, 48, 52, 54, 66, 77], 0], ['2022-08-11 10:46:48.099', [7, 18, 27, 39, 49, 53, 55, 68, 78], 0], ['2022-08-11 10:47:48.402', [8, 18, 27, 39, 49, 53, 55, 68, 79], 0], ['2022-08-11 10:48:48.548', [9, 18, 28, 40, 50, 53, 55, 69], 0], ['2022-08-11 10:49:49.820', [9, 19, 28, 41,

167 번째 시도:  [['2022-08-11 10:35:37.080', [11, 22, 32, 42, 50, 52, 60, 68], 0], ['2022-08-11 10:36:37.320', [11, 23, 32, 43, 51, 52, 60, 69], 0], ['2022-08-11 10:37:38.482', [12, 24, 33, 44, 51, 52, 61, 69], 0], ['2022-08-11 10:38:38.661', [12, 24, 33, 44, 51, 52, 62, 70], 0], ['2022-08-11 10:39:39.792', [13, 25, 34, 45, 51, 52, 63, 70], 0], ['2022-08-11 10:40:40.001', [14, 26, 34, 45, 52, 52, 63, 72], 0], ['2022-08-11 10:41:40.117', [3, 14, 26, 36, 46, 52, 53, 63, 74], 0], ['2022-08-11 10:42:43.357', [3, 15, 26, 36, 46, 52, 53, 64, 74], 0], ['2022-08-11 10:43:43.550', [4, 16, 26, 37, 47, 52, 53, 65, 74], 0], ['2022-08-11 10:44:46.759', [5, 16, 27, 37, 48, 52, 54, 65, 75], 0], ['2022-08-11 10:45:47.952', [6, 17, 27, 38, 48, 52, 54, 66, 77], 0], ['2022-08-11 10:46:48.099', [7, 18, 27, 39, 49, 53, 55, 68, 78], 0], ['2022-08-11 10:47:48.402', [8, 18, 27, 39, 49, 53, 55, 68, 79], 0], ['2022-08-11 10:48:48.548', [9, 18, 28, 40, 50, 53, 55, 69], 0], ['2022-08-11 10:49:49.820', [9, 19, 28, 41,

168 번째 시도:  [['2022-08-11 10:35:37.080', [11, 22, 32, 42, 50, 52, 60, 68], 0], ['2022-08-11 10:36:37.320', [11, 23, 32, 43, 51, 52, 60, 69], 0], ['2022-08-11 10:37:38.482', [12, 24, 33, 44, 51, 52, 61, 69], 0], ['2022-08-11 10:38:38.661', [12, 24, 33, 44, 51, 52, 62, 70], 0], ['2022-08-11 10:39:39.792', [13, 25, 34, 45, 51, 52, 63, 70], 0], ['2022-08-11 10:40:40.001', [14, 26, 34, 45, 52, 52, 63, 72], 0], ['2022-08-11 10:41:40.117', [3, 14, 26, 36, 46, 52, 53, 63, 74], 0], ['2022-08-11 10:42:43.357', [3, 15, 26, 36, 46, 52, 53, 64, 74], 0], ['2022-08-11 10:43:43.550', [4, 16, 26, 37, 47, 52, 53, 65, 74], 0], ['2022-08-11 10:44:46.759', [5, 16, 27, 37, 48, 52, 54, 65, 75], 0], ['2022-08-11 10:45:47.952', [6, 17, 27, 38, 48, 52, 54, 66, 77], 0], ['2022-08-11 10:46:48.099', [7, 18, 27, 39, 49, 53, 55, 68, 78], 0], ['2022-08-11 10:47:48.402', [8, 18, 27, 39, 49, 53, 55, 68, 79], 0], ['2022-08-11 10:48:48.548', [9, 18, 28, 40, 50, 53, 55, 69], 0], ['2022-08-11 10:49:49.820', [9, 19, 28, 41,

ConnectionError: HTTPConnectionPool(host='apis.data.go.kr', port=80): Max retries exceeded with url: /6410000/busarrivalservice/getBusArrivalItem?serviceKey=bwUUK6MEA6GnqDpo2ZS9ChN4VZ1nkUPXrryU80XxUcLqxB62pIGIPWXNdyzqgT20BHrw8zxWWk9rqiSQwTrbnQ%3D%3D&routeId=200000112&stationId=228000703 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001A807415CD0>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))

In [4]:
past=4
pBus=Bus(p7000)
for x in range(600): # 시간 설정
    soup=pBus.repet()
    try:
        result=int(soup.resultCode.text)
        arrived=0
        if past!=result:
            if result==0:
                arrived=1
        pBus.bus_time.append([soup.queryTime.text,pBus.current_buses(),arrived])
        past=result
        print(x,"번째 시도: ",pBus.bus_time)
        time.sleep(60)
    except Exception as e:
        continue

df=pd.DataFrame(pBus.bus_time,columns=['시간','운행중인 다른 버스들의 위치','arrived'])

mod_df=trim_data(df,'8.11.csv') # 이름 설정
mod_df

0 번째 시도:  [['2022-08-11 14:31:49.790', [11, 24, 34, 47, 57, 69, 74], 0]]
1 번째 시도:  [['2022-08-11 14:31:49.790', [11, 24, 34, 47, 57, 69, 74], 0], ['2022-08-11 14:32:49.967', [12, 25, 34, 48, 57, 70, 75], 0]]
5 번째 시도:  [['2022-08-11 14:31:49.790', [11, 24, 34, 47, 57, 69, 74], 0], ['2022-08-11 14:32:49.967', [12, 25, 34, 48, 57, 70, 75], 0], ['2022-08-11 14:33:51.614', [], 0]]
7 번째 시도:  [['2022-08-11 14:31:49.790', [11, 24, 34, 47, 57, 69, 74], 0], ['2022-08-11 14:32:49.967', [12, 25, 34, 48, 57, 70, 75], 0], ['2022-08-11 14:33:51.614', [], 0], ['2022-08-11 14:34:52.026', [], 0]]
8 번째 시도:  [['2022-08-11 14:31:49.790', [11, 24, 34, 47, 57, 69, 74], 0], ['2022-08-11 14:32:49.967', [12, 25, 34, 48, 57, 70, 75], 0], ['2022-08-11 14:33:51.614', [], 0], ['2022-08-11 14:34:52.026', [], 0], ['2022-08-11 14:35:52.189', [13, 26, 38, 50, 60, 72, 79], 0]]
10 번째 시도:  [['2022-08-11 14:31:49.790', [11, 24, 34, 47, 57, 69, 74], 0], ['2022-08-11 14:32:49.967', [12, 25, 34, 48, 57, 70, 75], 0], ['2022-08

22 번째 시도:  [['2022-08-11 14:31:49.790', [11, 24, 34, 47, 57, 69, 74], 0], ['2022-08-11 14:32:49.967', [12, 25, 34, 48, 57, 70, 75], 0], ['2022-08-11 14:33:51.614', [], 0], ['2022-08-11 14:34:52.026', [], 0], ['2022-08-11 14:35:52.189', [13, 26, 38, 50, 60, 72, 79], 0], ['2022-08-11 14:36:57.335', [15, 26, 39, 50, 61, 73, 79], 0], ['2022-08-11 14:38:04.684', [2, 15, 26, 39, 51, 62, 74], 1], ['2022-08-11 14:39:05.065', [3, 15, 27, 40, 51, 63, 74], 0], ['2022-08-11 14:40:11.320', [3, 16, 27, 40, 51, 63, 74], 0], ['2022-08-11 14:41:11.590', [4, 17, 27, 41, 52, 63, 75], 0], ['2022-08-11 14:42:12.752', [5, 18, 27, 41, 52, 64, 76], 0], ['2022-08-11 14:43:14.085', [6, 18, 28, 42, 52, 65, 78], 0], ['2022-08-11 14:44:16.277', [7, 19, 29, 42, 52, 65, 79], 0], ['2022-08-11 14:45:17.533', [8, 19, 29, 43, 53, 65], 0], ['2022-08-11 14:46:17.772', [9, 20, 29, 44, 53, 66], 0], ['2022-08-11 14:47:17.934', [9, 20, 30, 45, 54, 66], 0], ['2022-08-11 14:48:19.347', [9, 21, 31, 45, 54, 68], 0], ['2022-08-11 

30 번째 시도:  [['2022-08-11 14:31:49.790', [11, 24, 34, 47, 57, 69, 74], 0], ['2022-08-11 14:32:49.967', [12, 25, 34, 48, 57, 70, 75], 0], ['2022-08-11 14:33:51.614', [], 0], ['2022-08-11 14:34:52.026', [], 0], ['2022-08-11 14:35:52.189', [13, 26, 38, 50, 60, 72, 79], 0], ['2022-08-11 14:36:57.335', [15, 26, 39, 50, 61, 73, 79], 0], ['2022-08-11 14:38:04.684', [2, 15, 26, 39, 51, 62, 74], 1], ['2022-08-11 14:39:05.065', [3, 15, 27, 40, 51, 63, 74], 0], ['2022-08-11 14:40:11.320', [3, 16, 27, 40, 51, 63, 74], 0], ['2022-08-11 14:41:11.590', [4, 17, 27, 41, 52, 63, 75], 0], ['2022-08-11 14:42:12.752', [5, 18, 27, 41, 52, 64, 76], 0], ['2022-08-11 14:43:14.085', [6, 18, 28, 42, 52, 65, 78], 0], ['2022-08-11 14:44:16.277', [7, 19, 29, 42, 52, 65, 79], 0], ['2022-08-11 14:45:17.533', [8, 19, 29, 43, 53, 65], 0], ['2022-08-11 14:46:17.772', [9, 20, 29, 44, 53, 66], 0], ['2022-08-11 14:47:17.934', [9, 20, 30, 45, 54, 66], 0], ['2022-08-11 14:48:19.347', [9, 21, 31, 45, 54, 68], 0], ['2022-08-11 

36 번째 시도:  [['2022-08-11 14:31:49.790', [11, 24, 34, 47, 57, 69, 74], 0], ['2022-08-11 14:32:49.967', [12, 25, 34, 48, 57, 70, 75], 0], ['2022-08-11 14:33:51.614', [], 0], ['2022-08-11 14:34:52.026', [], 0], ['2022-08-11 14:35:52.189', [13, 26, 38, 50, 60, 72, 79], 0], ['2022-08-11 14:36:57.335', [15, 26, 39, 50, 61, 73, 79], 0], ['2022-08-11 14:38:04.684', [2, 15, 26, 39, 51, 62, 74], 1], ['2022-08-11 14:39:05.065', [3, 15, 27, 40, 51, 63, 74], 0], ['2022-08-11 14:40:11.320', [3, 16, 27, 40, 51, 63, 74], 0], ['2022-08-11 14:41:11.590', [4, 17, 27, 41, 52, 63, 75], 0], ['2022-08-11 14:42:12.752', [5, 18, 27, 41, 52, 64, 76], 0], ['2022-08-11 14:43:14.085', [6, 18, 28, 42, 52, 65, 78], 0], ['2022-08-11 14:44:16.277', [7, 19, 29, 42, 52, 65, 79], 0], ['2022-08-11 14:45:17.533', [8, 19, 29, 43, 53, 65], 0], ['2022-08-11 14:46:17.772', [9, 20, 29, 44, 53, 66], 0], ['2022-08-11 14:47:17.934', [9, 20, 30, 45, 54, 66], 0], ['2022-08-11 14:48:19.347', [9, 21, 31, 45, 54, 68], 0], ['2022-08-11 

41 번째 시도:  [['2022-08-11 14:31:49.790', [11, 24, 34, 47, 57, 69, 74], 0], ['2022-08-11 14:32:49.967', [12, 25, 34, 48, 57, 70, 75], 0], ['2022-08-11 14:33:51.614', [], 0], ['2022-08-11 14:34:52.026', [], 0], ['2022-08-11 14:35:52.189', [13, 26, 38, 50, 60, 72, 79], 0], ['2022-08-11 14:36:57.335', [15, 26, 39, 50, 61, 73, 79], 0], ['2022-08-11 14:38:04.684', [2, 15, 26, 39, 51, 62, 74], 1], ['2022-08-11 14:39:05.065', [3, 15, 27, 40, 51, 63, 74], 0], ['2022-08-11 14:40:11.320', [3, 16, 27, 40, 51, 63, 74], 0], ['2022-08-11 14:41:11.590', [4, 17, 27, 41, 52, 63, 75], 0], ['2022-08-11 14:42:12.752', [5, 18, 27, 41, 52, 64, 76], 0], ['2022-08-11 14:43:14.085', [6, 18, 28, 42, 52, 65, 78], 0], ['2022-08-11 14:44:16.277', [7, 19, 29, 42, 52, 65, 79], 0], ['2022-08-11 14:45:17.533', [8, 19, 29, 43, 53, 65], 0], ['2022-08-11 14:46:17.772', [9, 20, 29, 44, 53, 66], 0], ['2022-08-11 14:47:17.934', [9, 20, 30, 45, 54, 66], 0], ['2022-08-11 14:48:19.347', [9, 21, 31, 45, 54, 68], 0], ['2022-08-11 

...

84 번째 시도:  [['2022-08-11 14:31:49.790', [11, 24, 34, 47, 57, 69, 74], 0], ['2022-08-11 14:32:49.967', [12, 25, 34, 48, 57, 70, 75], 0], ['2022-08-11 14:33:51.614', [], 0], ['2022-08-11 14:34:52.026', [], 0], ['2022-08-11 14:35:52.189', [13, 26, 38, 50, 60, 72, 79], 0], ['2022-08-11 14:36:57.335', [15, 26, 39, 50, 61, 73, 79], 0], ['2022-08-11 14:38:04.684', [2, 15, 26, 39, 51, 62, 74], 1], ['2022-08-11 14:39:05.065', [3, 15, 27, 40, 51, 63, 74], 0], ['2022-08-11 14:40:11.320', [3, 16, 27, 40, 51, 63, 74], 0], ['2022-08-11 14:41:11.590', [4, 17, 27, 41, 52, 63, 75], 0], ['2022-08-11 14:42:12.752', [5, 18, 27, 41, 52, 64, 76], 0], ['2022-08-11 14:43:14.085', [6, 18, 28, 42, 52, 65, 78], 0], ['2022-08-11 14:44:16.277', [7, 19, 29, 42, 52, 65, 79], 0], ['2022-08-11 14:45:17.533', [8, 19, 29, 43, 53, 65], 0], ['2022-08-11 14:46:17.772', [9, 20, 29, 44, 53, 66], 0], ['2022-08-11 14:47:17.934', [9, 20, 30, 45, 54, 66], 0], ['2022-08-11 14:48:19.347', [9, 21, 31, 45, 54, 68], 0], ['2022-08-11 

86 번째 시도:  [['2022-08-11 14:31:49.790', [11, 24, 34, 47, 57, 69, 74], 0], ['2022-08-11 14:32:49.967', [12, 25, 34, 48, 57, 70, 75], 0], ['2022-08-11 14:33:51.614', [], 0], ['2022-08-11 14:34:52.026', [], 0], ['2022-08-11 14:35:52.189', [13, 26, 38, 50, 60, 72, 79], 0], ['2022-08-11 14:36:57.335', [15, 26, 39, 50, 61, 73, 79], 0], ['2022-08-11 14:38:04.684', [2, 15, 26, 39, 51, 62, 74], 1], ['2022-08-11 14:39:05.065', [3, 15, 27, 40, 51, 63, 74], 0], ['2022-08-11 14:40:11.320', [3, 16, 27, 40, 51, 63, 74], 0], ['2022-08-11 14:41:11.590', [4, 17, 27, 41, 52, 63, 75], 0], ['2022-08-11 14:42:12.752', [5, 18, 27, 41, 52, 64, 76], 0], ['2022-08-11 14:43:14.085', [6, 18, 28, 42, 52, 65, 78], 0], ['2022-08-11 14:44:16.277', [7, 19, 29, 42, 52, 65, 79], 0], ['2022-08-11 14:45:17.533', [8, 19, 29, 43, 53, 65], 0], ['2022-08-11 14:46:17.772', [9, 20, 29, 44, 53, 66], 0], ['2022-08-11 14:47:17.934', [9, 20, 30, 45, 54, 66], 0], ['2022-08-11 14:48:19.347', [9, 21, 31, 45, 54, 68], 0], ['2022-08-11 

90 번째 시도:  [['2022-08-11 14:31:49.790', [11, 24, 34, 47, 57, 69, 74], 0], ['2022-08-11 14:32:49.967', [12, 25, 34, 48, 57, 70, 75], 0], ['2022-08-11 14:33:51.614', [], 0], ['2022-08-11 14:34:52.026', [], 0], ['2022-08-11 14:35:52.189', [13, 26, 38, 50, 60, 72, 79], 0], ['2022-08-11 14:36:57.335', [15, 26, 39, 50, 61, 73, 79], 0], ['2022-08-11 14:38:04.684', [2, 15, 26, 39, 51, 62, 74], 1], ['2022-08-11 14:39:05.065', [3, 15, 27, 40, 51, 63, 74], 0], ['2022-08-11 14:40:11.320', [3, 16, 27, 40, 51, 63, 74], 0], ['2022-08-11 14:41:11.590', [4, 17, 27, 41, 52, 63, 75], 0], ['2022-08-11 14:42:12.752', [5, 18, 27, 41, 52, 64, 76], 0], ['2022-08-11 14:43:14.085', [6, 18, 28, 42, 52, 65, 78], 0], ['2022-08-11 14:44:16.277', [7, 19, 29, 42, 52, 65, 79], 0], ['2022-08-11 14:45:17.533', [8, 19, 29, 43, 53, 65], 0], ['2022-08-11 14:46:17.772', [9, 20, 29, 44, 53, 66], 0], ['2022-08-11 14:47:17.934', [9, 20, 30, 45, 54, 66], 0], ['2022-08-11 14:48:19.347', [9, 21, 31, 45, 54, 68], 0], ['2022-08-11 

92 번째 시도:  [['2022-08-11 14:31:49.790', [11, 24, 34, 47, 57, 69, 74], 0], ['2022-08-11 14:32:49.967', [12, 25, 34, 48, 57, 70, 75], 0], ['2022-08-11 14:33:51.614', [], 0], ['2022-08-11 14:34:52.026', [], 0], ['2022-08-11 14:35:52.189', [13, 26, 38, 50, 60, 72, 79], 0], ['2022-08-11 14:36:57.335', [15, 26, 39, 50, 61, 73, 79], 0], ['2022-08-11 14:38:04.684', [2, 15, 26, 39, 51, 62, 74], 1], ['2022-08-11 14:39:05.065', [3, 15, 27, 40, 51, 63, 74], 0], ['2022-08-11 14:40:11.320', [3, 16, 27, 40, 51, 63, 74], 0], ['2022-08-11 14:41:11.590', [4, 17, 27, 41, 52, 63, 75], 0], ['2022-08-11 14:42:12.752', [5, 18, 27, 41, 52, 64, 76], 0], ['2022-08-11 14:43:14.085', [6, 18, 28, 42, 52, 65, 78], 0], ['2022-08-11 14:44:16.277', [7, 19, 29, 42, 52, 65, 79], 0], ['2022-08-11 14:45:17.533', [8, 19, 29, 43, 53, 65], 0], ['2022-08-11 14:46:17.772', [9, 20, 29, 44, 53, 66], 0], ['2022-08-11 14:47:17.934', [9, 20, 30, 45, 54, 66], 0], ['2022-08-11 14:48:19.347', [9, 21, 31, 45, 54, 68], 0], ['2022-08-11 

ConnectionError: HTTPConnectionPool(host='apis.data.go.kr', port=80): Max retries exceeded with url: /6410000/busarrivalservice/getBusArrivalItem?serviceKey=bwUUK6MEA6GnqDpo2ZS9ChN4VZ1nkUPXrryU80XxUcLqxB62pIGIPWXNdyzqgT20BHrw8zxWWk9rqiSQwTrbnQ%3D%3D&routeId=200000112&stationId=228000703 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001A807404B50>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))